In [1]:
!pip install opencv-python deepface numpy pandas

In [2]:
!pip install tf-keras --upgrade
!pip install deepface --upgrade

In [3]:
import os
import cv2
import numpy as np
import pandas as pd
from datetime import datetime
from deepface import DeepFace

In [4]:
class FaceDatabase:
    def __init__(self):
        self.known_faces = {}  # Format: {"Name": [img_path1, img_path2]}
    
    def load_from_folder(self, folder_path="known_faces"):
        """Load all images from subfolders (1 subfolder per person)"""
        for person_name in os.listdir(folder_path):
            person_dir = os.path.join(folder_path, person_name)
            if os.path.isdir(person_dir):
                image_paths = []
                for img_file in os.listdir(person_dir):
                    if img_file.lower().endswith((".jpg", ".jpeg", ".png")):
                        image_paths.append(os.path.join(person_dir, img_file))
                if image_paths:
                    self.known_faces[person_name] = image_paths
                    print(f"Loaded {len(image_paths)} images for {person_name}")

# Initialize database
face_db = FaceDatabase()
face_db.load_from_folder()

Loaded 6 images for Dhanuka
Loaded 16 images for Dilasha
Loaded 6 images for Dimalka
Loaded 11 images for Hansith
Loaded 7 images for Irantha
Loaded 8 images for Kalindu
Loaded 3 images for Ranshika
Loaded 1 images for Ravindu
Loaded 8 images for Sachini
Loaded 15 images for Sakun
Loaded 10 images for Thineesha
Loaded 14 images for Thisaru
Loaded 15 images for Veruni
Loaded 6 images for Vidumini


In [5]:
class AttendanceSystem:
    def __init__(self, face_db):
        self.face_db = face_db
        self.attendance_log = []
    
    def identify_person(self, frame):
        """Identify who is in the frame"""
        try:
            temp_path = "temp_frame.jpg"
            cv2.imwrite(temp_path, frame)
            
            for name, img_paths in self.face_db.known_faces.items():
                for img_path in img_paths:
                    result = DeepFace.verify(
                        img1_path=img_path,
                        img2_path=temp_path,
                        model_name="Facenet", # Pre-trained Facenet model
                        enforce_detection=False, # Skips if no face detected
                        detector_backend="opencv", # Uses OpenCV for face detection
                        distance_metric="cosine" # Measures similarity between embeddings
                    )
                    if result["verified"]:
                        os.remove(temp_path)
                        return name
            os.remove(temp_path)
        except Exception as e:
            print(f"Error: {e}")
        return None
    
    def mark_attendance(self, name):
        timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        self.attendance_log.append([name, timestamp])
        print(f"Attendance marked for {name} at {timestamp}")
    
    def run_once_and_close(self):
        """Capture once, identify, mark attendance, and exit"""
        cap = cv2.VideoCapture(0)
        try:
            ret, frame = cap.read()
            if not ret:
                print("Camera error!")
                return
            
            # Display captured frame briefly
            cv2.imshow("Identifying...", frame)
            cv2.waitKey(500)  # Show for 0.5 seconds
            
            name = self.identify_person(frame)
            if name:
                self.mark_attendance(name)
            else:
                print("No recognized face found")
                
        finally:
            cap.release()
            cv2.destroyAllWindows()
            self.save_attendance()
    
    def save_attendance(self, filename="attendance.csv"):
        df = pd.DataFrame(self.attendance_log, columns=["Name", "Timestamp"])
        df.to_csv(filename, mode='a', header=not os.path.exists(filename), index=False)
        print(f"Attendance saved to {filename}")

In [6]:
# ===== EXECUTION =====
attendance_system = AttendanceSystem(face_db)
attendance_system.run_once_and_close()  # Single detection and exit

Attendance marked for Dhanuka at 2025-04-12 14:27:02
Attendance saved to attendance.csv


In [7]:
class SystemTester:
    def __init__(self, attendance_system):
        self.system = attendance_system
    
    def run_test(self, test_folder="test_samples"):
        results = []
        
        # Test known faces
        known_dir = os.path.join(test_folder, "known")
        for img_file in os.listdir(known_dir):
            img_path = os.path.join(known_dir, img_file)
            frame = cv2.imread(img_path)
            name = self.system.identify_person(frame)
            results.append(("Known", name is not None))  # Should return your name
            
        # Test unknown samples
        unknown_dir = os.path.join(test_folder, "unknown")
        for img_file in os.listdir(unknown_dir):
            img_path = os.path.join(unknown_dir, img_file)
            frame = cv2.imread(img_path)
            name = self.system.identify_person(frame)
            results.append(("Unknown", name is None))  # Should return None
            
        # Calculate metrics
        true_pos = sum(1 for x in results if x[0]=="Known" and x[1])
        true_neg = sum(1 for x in results if x[0]=="Unknown" and x[1])
        
        accuracy = (true_pos + true_neg) / len(results)
        print(f"""
        Accuracy Report:
        - True Positives: {true_pos}/{len(os.listdir(known_dir))}
        - True Negatives: {true_neg}/{len(os.listdir(unknown_dir))}
        - Overall Accuracy: {accuracy:.1%}
        """)

In [8]:
# After initializing your system
tester = SystemTester(attendance_system)
tester.run_test()


        Accuracy Report:
        - True Positives: 4/6
        - True Negatives: 3/6
        - Overall Accuracy: 58.3%
        
